# Parse XML annotation file with X,Y coordinates and instance ID into a DataFrame

In [1]:
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd

In [2]:
def xml_to_df(xml_filepath):

    tree = ET.parse(xml_filepath)
    root = tree.getroot()
    classlist = []
    for Annotation in root.iter("Annotation"):
        for Attribute in Annotation.iter("Attribute"):
            classlist.append(Attribute.attrib.get('Name'))
    classlists = sorted(classlist)
    append_df = []
    for index, Annotation in enumerate(root.iter("Annotation")):
        for Region in Annotation.iter('Region'): #iterate over the Region so we can iterate over id 1 and 2 (two circles):
            x = np.array([Vertex.get('X') for Vertex in Region.iter('Vertex')])
            y = np.array([Vertex.get('Y') for Vertex in Region.iter('Vertex')])
            id = np.array([int(Region.get('Id'))])
            classnames = classlists[index]
            coord_dict = {"ClassNames": [classnames], "X": [x], "Y": [y], "ID": [id]}
            df = pd.DataFrame(data = coord_dict)
            df.ID = df.ID.astype(int)
            append_df.append(df)
    coord_df = pd.concat(append_df).reset_index(drop=True)
    return(coord_df)

In [3]:
coord_df = xml_to_df(r'\\fatherserverdw\kyuex\clue images\annotations\roi\2022-06-07 13.18.40.xml')
coord_df

,ClassNames,X,Y,ID
0,tissue,"[5615, 5725, 5810, 5865, 5900, 5915, 5930, 594...","[10850, 10890, 10930, 10950, 10960, 10970, 109...",1
1,tissue,"[24299, 24364, 24483, 24537, 24613, 24624, 246...","[20025, 20068, 20133, 20176, 20209, 20231, 202...",2


# Convert X,Y Coordinates to Binary Mask and Read and Resize Image:

In [4]:
import openslide
import os
os.environ["OPENCV_IO_MAX_IMAGE_PIXELS"] = pow(2,40).__str__()
import cv2
from matplotlib import pyplot as plt
def df_to_image_mask(xml_filepath, ndpi_filepath,downsample_factor):
    coord_df = xml_to_df(xml_filepath)
    slide = openslide.open_slide(ndpi_filepath)
    rgb_dim = slide.dimensions
    print("Dimension of level 0 of the image is: ",rgb_dim)
    slide_level_dim = slide.level_dimensions
    num_levels = len(slide_level_dim)
    print("Number of levels in this image are:",num_levels)
    factors = slide.level_downsamples
    print("Each level is downsampled by:",factors)
    blank = np.zeros(rgb_dim, dtype = np.uint8) #white
    for idx, row in coord_df.iterrows():
        xx = row.X
        yy = row.Y
        xy = list(zip(xx,yy))
        contours = np.array(xy)
        mask = cv2.fillPoly(blank, pts=[contours.astype(int)], color=idx+1)
    target_level = slide.get_best_level_for_downsample(downsample_factor)
    target_dim = slide.level_dimensions[target_level]
    rsf = [x/y for x,y in zip(rgb_dim,target_dim)]
    image_resized = slide.read_region(location=(0,0),level=target_level,size=target_dim)
    mask_resized = cv2.resize(mask,[_//round(rsf[0]) for _ in mask.shape],interpolation=cv2.INTER_NEAREST)
    return image_resized,mask_resized

# Cropping RGB Image with Mask:

In [5]:
image,mask = df_to_image_mask(r'\\fatherserverdw\kyuex\clue images\annotations\roi\2022-06-07 13.18.40.xml',r'\\fatherserverdw\kyuex\clue images\2022-06-07 13.18.40.ndpi',10)

Dimension of level 0 of the image is:  (48000, 47872)
Number of levels in this image are: 8
Each level is downsampled by: (1.0, 2.0, 4.0, 8.0, 16.0, 32.0, 64.0, 128.0)


In [6]:
def crop_final_mask(image,mask):
    imagearr = np.array(image)
    imagearr = imagearr[:,:,:3]
    maskarr = np.array(mask)
    maskrgb = np.repeat(maskarr[:,:,np.newaxis],3,axis=2)
    for tissueID in range(1,np.max(maskrgb[:])): #from first tissue id to end
        #masking
        masktmp = mask==tissueID #boolean
        masktmp = np.repeat(masktmp[:,:,np.newaxis],3,axis=2) #change shape to match shape of imagearr
        final_image = np.multiply(imagearr,masktmp)
        #crop
        [x_crop,y_crop] = np.where(final_image[:,:,0]>0)
        cropped_final_image = final_image[np.min(y_crop):np.max(y_crop),np.min(x_crop):np.max(x_crop)]
        cropped_final_image = cropped_final_image[:,:,::-1]
        return cropped_final_image
        #cv2.imwrite(r"\\fatherserverdw\Kevin\\" + "final_image" + str(tissueID) + ".jpg",cropped_final_image)

In [7]:
crop_final_mask(image,mask)

array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)

# Now create mask for multiple classes:

In [8]:
# coord_df_1 = xml_to_df(r'\\fatherserverdw\kyuex\clue images\annotations\12class\unhealthy_wrongbody\2022-06-07 13.18.40.xml')
# coord_df_1 = coord_df_1.drop(columns = ["ID"])
# dict = {"corneum" : 1,"spinosum": 2,"hairshaft":3,"hairfollicle":4,"smoothmuscle":5,"oil":6,"sweat":7,"nerve":8,"bloodvessel":9,"ecm":10,"fat":11,"white":12}
# coord_df_1 = coord_df_1.replace({"ClassNames": dict})
# coord_df_1.to_pickle(r'\\fatherserverdw\Kevin\coord_df.pkl')

coord_df_1 = pd.read_pickle(r'\\fatherserverdw\Kevin\coord_df.pkl')
coord_df_1

,ClassNames,X,Y
0,9,"[7833, 7833, 7833, 7834, 7834, 7835, 7835, 783...","[6594, 6593, 6592, 6592, 6591, 6591, 6589, 658..."
1,9,"[8596, 8597, 8597, 8598, 8598, 8600, 8600, 860...","[6019, 6019, 6018, 6015, 6014, 6014, 6013, 601..."
2,9,"[10214, 10215, 10215, 10215, 10216, 10216, 102...","[4918, 4918, 4915, 4914, 4914, 4913, 4913, 491..."
3,9,"[8993, 8992, 8991, 8990, 8990, 8989, 8988, 898...","[5313, 5313, 5313, 5313, 5315, 5315, 5315, 531..."
4,9,"[7521, 7521, 7521, 7521, 7521, 7522, 7522, 752...","[7007, 7006, 7004, 7003, 7002, 7002, 7001, 700..."
...,...,...,...
144,12,"[33402, 33404, 33405, 33406, 33406, 33407, 334...","[20590, 20590, 20590, 20590, 20591, 20591, 205..."
145,12,"[39948, 39955, 39959, 39970, 39981, 39989, 400...","[39749, 39749, 39749, 39749, 39749, 39752, 397..."
146,12,"[39193, 39189, 39185, 39181, 39178, 39178, 391...","[37963, 37963, 37974, 37978, 37982, 37986, 379..."
147,12,"[39767, 39769, 39770, 39772, 39773, 39774, 397...","[38084, 38084, 38084, 38085, 38085, 38086, 380..."


In [ ]:
# for label in range(nlabels):
#     dflabel = coord_df_1.iloc[coord_df_1[coord_df_1.ClassLabels == label].index[0]:coord_df_1[coord_df_1.ClassLabels == label].index[-1]]

slide = openslide.open_slide(r'\\fatherserverdw\kyuex\clue images\2022-06-07 13.18.40.ndpi')

for idx, row in coord_df_1.iterrows():
    xx = row.X.astype('int')
    yy = row.Y.astype('int')
    rgb_dim = (int(min(xx)),int(min(yy)))
    print(rgb_dim)
    blank = np.zeros(rgb_dim, dtype = np.uint8)
    print(blank.shape)
    xy = list(zip(xx-rgb_dim[0],yy-rgb_dim[1]))
    contours = np.array(xy)
    print(contours)
    class_number = coord_df_1.ClassNames.values[idx]
    mask = cv2.fillPoly(blank, pts=[contours.astype(int)], color=int(class_number+1))
    plt.imshow(mask)
    #problem of blank shape being much larger than the contours...fix so blank shape is similar to rgb_dim
# rsf = [x/y for x,y in zip(rgb_dim,target_dim)]
# image_resized = slide.read_region(location=(0,0),level=target_level,size=target_dim)
# mask_resized = cv2.resize(mask,[_//round(rsf[0]) for _ in mask.shape],interpolation=cv2.INTER_NEAREST)
# plt.imshow(mask_resized)

(7775, 6413)
[[ 58 181]
 [ 58 180]
 [ 58 179]
 ...
 [ 59 182]
 [ 59 181]
 [ 58 181]]
(8561, 5806)
[[ 35 213]
 [ 36 213]
 [ 36 212]
 ...
 [ 25 213]
 [ 26 213]
 [ 35 213]]
(10180, 4701)
[[ 34 217]
 [ 35 217]
 [ 35 214]
 ...
 [ 26 225]
 [ 27 225]
 [ 34 217]]
(8881, 5287)
[[112  26]
 [111  26]
 [110  26]
 ...
 [114  25]
 [112  25]
 [112  26]]
(7463, 6900)
[[ 58 107]
 [ 58 106]
 [ 58 104]
 ...
 [ 59 109]
 [ 59 108]
 [ 58 107]]
(31203, 6253)
[[332  36]
 [330  37]
 [330  40]
 ...
 [333  34]
 [333  35]
 [332  36]]
(30847, 6622)
[[186  48]
 [185  51]
 [185  52]
 ...
 [186  46]
 [186  47]
 [186  48]]
(30672, 6879)
[[71 27]
 [71 29]
 [70 29]
 ...
 [73 25]
 [71 25]
 [71 27]]
(30407, 7222)
[[  5 174]
 [  5 175]
 [  7 175]
 ...
 [  7 169]
 [  7 170]
 [  5 174]]
(30215, 7476)
[[138  19]
 [138  20]
 [137  20]
 ...
 [136   5]
 [137   8]
 [138  19]]
(14812, 20809)
[[138 102]
 [137 102]
 [136 102]
 ...
 [142 106]
 [141 106]
 [138 102]]
(13692, 21143)
[[230  84]
 [229  84]
 [228  84]
 ...
 [235  78]
 [235

In [37]:
ra = np.array(mask)
ra

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [19]:
np.unique(ra,return_counts = True)

(array([0], dtype=uint8), array([33675264], dtype=int64))